In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

email = "yong2khoo@hotmail.com"
password = "y2k@linkedin"
driver.get("https://www.linkedin.com/login")
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username")))

email_elem = driver.find_element(By.ID, "username")
email_elem.send_keys(email)

password_elem = driver.find_element(By.ID, "password")
password_elem.send_keys(password)
password_elem.submit()

driver.implicitly_wait(3)

In [ ]:
pages = range(25)
links = []
for page in pages:
    url_search_by_people = f"https://www.linkedin.com/search/results/people/?keywords=Data%20Scientist&origin=SWITCH_SEARCH_VERTICAL&page={page+15}"
    driver.get(url_search_by_people)
    elems = driver.find_elements(By.CSS_SELECTOR, ".mb1 a.app-aware-link")
    for elem in elems:
        link = elem.get_attribute('href').split("?")[0]
        if link != 'https://www.linkedin.com/search/results/people/headless':
            links.append(link)


In [ ]:
print(links)

In [ ]:
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd 
from selenium.webdriver.common.by import By

#i try avoid list comprehension here, fearing it adds overhead
candidates = []
driver.implicitly_wait(2)
for link in links:
    print(f"processing {link}")
    driver.get(link)
    
    elem_name = driver.find_element(By.CLASS_NAME, "text-heading-xlarge").text
    elem_title = driver.find_element(By.CSS_SELECTOR, "div.pv-text-details__left-panel > div.text-body-medium").text
    elem_loc = driver.find_element(By.CSS_SELECTOR, "div.pv-text-details__left-panel > span.text-body-small").text
    
    es_ = driver.find_elements(By.CSS_SELECTOR, "section.artdeco-card.ember-view.relative.break-words")
    
    time.sleep(1)    
    ###Experience###
    section_exp = []
    for e_ in es_:
        div_exps = e_.find_elements(By.CSS_SELECTOR, "div#experience")
        if len(div_exps) > 0:
            section_exp.append(e_)
            break
    
    if len(section_exp) > 0:
        elem_exp = section_exp[0]
        elems_exp = elem_exp.find_elements(By.CSS_SELECTOR, "div.pvs-list__outer-container div.display-flex.flex-column.full-width.align-self-center")

        exps = []
        for elem_exp in elems_exp:
            spans = elem_exp.find_elements(By.CSS_SELECTOR, "span.t-14 > span")
            exp = [span.text for span in spans if span.get_attribute("aria-hidden") is not None]
            
            title_spans = elem_exp.find_elements(By.CSS_SELECTOR, "span.mr1 > span")
            titles = [span.text for span in title_spans if span.get_attribute("aria-hidden") is not None]
            exps.append({"titles":titles, "exp": exp})
    ##################
    
    ###Education###
    section_edu = []
    for e_ in es_:
        div_edus = e_.find_elements(By.CSS_SELECTOR, "div#education")
        if len(div_edus) > 0:
            section_edu.append(e_)
            break
    
    if len(section_edu) > 0:
        elem_edu = section_edu[0]
        elems_edu = elem_edu.find_elements(By.CSS_SELECTOR, "div.pvs-list__outer-container div.display-flex.flex-column.full-width.align-self-center")

        edus = []
        for elem_edu in elems_edu:
            spans = elem_edu.find_elements(By.CSS_SELECTOR, "span.t-14 > span")
            edu = [span.text for span in spans if span.get_attribute("aria-hidden") is not None]
            edus.append(edu)
            
            
    ###Skills###
    skill_page_url = f"{link}/details/skills"
    print(skill_page_url)
    driver.get(skill_page_url)
    time.sleep(2)    
   
    tt = driver.find_elements(By.CSS_SELECTOR, "section.artdeco-card.ember-view")
    if len(tt) > 0:
        span_skills = tt[0].find_elements(By.CSS_SELECTOR, "div.display-flex.align-items-center span.mr1 > span")
        skills = list(set([span.text for span in span_skills if span.get_attribute("aria-hidden") is not None]))
        skills = [skill for skill in skills if skill]
        
    candidate = {"link": link, "name": elem_name, "current_title": elem_title, "current_location": elem_loc, 
                           "exps": exps, "edus": edus, "skills": skills}
    
    df = pd.DataFrame([candidate])
    df.to_csv('full-all-2.csv', encoding="utf-16", header=False, mode='a', index=False)
    
    candidates.append(candidate)
    
    
print(f"Done processing {len(candidates)} candidates ({count})")

In [ ]:
#Debugging if necessary
print(candidates)

###### IGNORE CELLS BELOW#############

#person_url = "https://www.linkedin.com/in/yong-yong-khoo-6597a3137/"
#person_url2 = "https://www.linkedin.com/in/joey-sham-aa2a50122/"

#s_input = driver.find_element_by_css_selector("#global-nav-typeahead > input")
#s_input.send_keys("Data Scientist")
#s_input.send_keys(Keys.ENTER)

#driver.get(links[2])
#driver.implicitly_wait(10)

#elems_with_artdeco = driver.find_elements_by_css_selector("section.artdeco-card.ember-view.relative.break-words")
#print(len(elems))
#print(elem_by_xpath)
#print(elem_by_xpath.get_attribute("innerHTML"))
#print(elems_exp.get_attribute("innerHTML"))
#div.display-flex.flex-column.full-width

In [ ]:
    """   
    #Section 4 = Experience
    time.sleep(1)
    exp_xpath = "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[4]"
    exp_xpath = "/html/body/div[4]/div[3]/div/div/div[2]/div/div/main/section[5]"
    #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "experience")))
    
    elem_by_xpath = driver.find_element_by_xpath(exp_xpath)
    elems_exp = elem_by_xpath.find_elements_by_css_selector("div.pvs-list__outer-container div.display-flex.flex-column.full-width.align-self-center")
    exps = []
    for elem_exp in elems_exp:
        spans = elem_exp.find_elements_by_css_selector("span.t-14 > span")
        exp = [span.text for span in spans if span.get_attribute("aria-hidden") is not None]
        exps.append(exp)
     
    #Section 5 = Education
    elem_by_xpath = driver.find_element_by_xpath("/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[5]")
    elems_edus = elem_by_xpath.find_elements_by_css_selector("div.pvs-list__outer-container div.display-flex.flex-column.full-width.align-self-center")
    edus = []
    for elem_edu in elems_edus:
        spans = elem_exp.find_elements_by_css_selector("span.t-14 > span")
        edu = [span.text for span in spans if span.get_attribute("aria-hidden") is not None]
        edus.append(edu)
    
    
    """
     '''
    section_skill = []
    for e_ in es_:
        div_skills = e_.find_elements_by_css_selector("div#skills")
        if len(div_skills) > 0:
            section_skill.append(e_)
            break
    
    if len(section_skill) > 0:
        elem_skill = section_skill[0]
        elems_skills = elem_skill.find_elements_by_css_selector("div.pvs-list__outer-container div.display-flex.flex-column.full-width.align-self-center")

        skills = []
        for elem_skill in elems_skills:
            spans = elem_skill.find_elements_by_css_selector("span.mr1 > span")
            skill = [span.text for span in spans if span.get_attribute("aria-hidden") is not None]
            skills.append(skill)

    candidate = {"link": link, "name": elem_name, "current_title": elem_title, "current_location": elem_loc, 
                           "exps": exps, "edus": edus, "skills": skills}
    candidates.append(candidate)
    '''